# <font color=red>Submission of VAT return to Altinn3</font>

Test-script to file a VAT return to the app on Altinn3, which is running on the tt02 environment on Altinn Studio.

## <font color=red>Check that required libraries are installed</font>

In [ ]:
try:
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from Steg import load_xml_files, get_altinntoken, get_instance, upload_vatreturnsubmission.py, upload_vatreturn
    from Steg import upload_attachment, log_in_idporten, update_process, create_instance, delete_attachment
    from Steg.SubmissionServices import Environment, LoginMethod, printing, App
    from Steg.validation import validate_vat_return, validate_example_file
except ImportError as e:
    try: 
        print("Missing dependencies, installing with pip")
        !pip install python-jose[cryptography] cryptography
        !pip install xmltodict
        !pip install requests
        import xmltodict
        import requests
        from Steg import load_xml_files, get_altinntoken, get_instance, upload_vatreturnsubmission.py, upload_vatreturn
        from Steg import upload_attachment, log_in_idporten, update_process, create_instance, delete_attachment
        from Steg.SubmissionServices import Environment, LoginMethod, printing, App
        from Steg.validation import validate_vat_return, validate_example_file
    except ImportError as err:
        print("Missing dependencies, installing with pip")
        !python3 -m pip install python-jose[cryptography] cryptography
        !python3 -m pip install 'python-jose[cryptography]'
        !python3 -m pip install xmltodict
        !python3 -m pip install requests
        import xmltodict
        import requests
        from Steg import load_xml_files, get_altinntoken, get_instance, upload_vatreturnsubmission.py, upload_vatreturn
        from Steg import upload_attachment, log_in_idporten, update_process, create_instance, delete_attachment
        from Steg.SubmissionServices import Environment, LoginMethod, printing, App
        from Steg.validation import validate_vat_return, validate_example_file

## <font color=red>Define variables</font>

In [ ]:
delete_previous_attachment = False  # If you want to delete an attachment you've uploaded by mistake.
validate = True  # If you want to run through the validation.

# No need to touch these
environment = Environment.tt02  # tt02
login_method = LoginMethod.idporten  # idporten
domain = environment.value  # Domain to the app on Altinn Studio
app = App.ETM2.value  # Which app repository that is running

# File name for VAT Return Submission ("envelope") and VAT Return that is going to be uploaded.
# Located in ./eksempler/konvolutt/ and ./eksempler/melding/
VatReturnSubmission_filename = "mvakonvolutt1.xml"
VatReturn_filename = "mvakode3.xml"

header = None

# Insert organization number you wish to send in for.
# Make note that the person that is going to file needs to have the appropriate roles and rights.
org_number = "123456789"

## <font color=red>Define VAT Return and VAT Return Submission</font>

In [ ]:
vat_return_xml, envelope_xml = load_xml_files.get(VatReturnSubmission_filename, VatReturn_filename)

## <font color=red>Generate ID-porten token</font>
The token is valid for 300 seconds, re-run this part if you haven't reached the altinntoken in time.

In [ ]:
header = log_in_idporten.get_idtoken()

## <font color=red>Validate message(Optional)</font>

In [ ]:
if login_method == validate:
  validation = validate_vat_return(dict(header), xml=vat_return_xml)

## <font color=red>Get AltinnToken</font>

In [ ]:
altinn_token = get_altinntoken.get(header)

## <font color=red>Create new instance</font>

In [ ]:
instance = create_instance.create(domain, altinn_token, app, org_number)

partyId = instance['instanceOwner']['partyId']
instanceGuid = instance['data'][0]['instanceGuid']
dataId = instance['data'][0]['id']

instanceUrl = domain + instance['appId'] + "/instances/" + partyId + "/" + instanceGuid
VATReturnSubmissionURL  = instanceUrl + "/data/" + dataId
VATReturnURL = instanceUrl + "/data?dataType=mvamelding"
VATREturnAttachmentURL  = instanceUrl + "/data?dataType=vedlegg"

## <font color=red>Upload VAT Return Submission</font>

In [ ]:
upload_vatreturnsubmission.upload(VATReturnSubmissionURL, envelope_xml, altinn_token)

## <font color=red>Upload VAT Return</font>

In [ ]:
upload_vatreturn.upload(VATReturnURL, vat_return_xml, altinn_token)

## <font color=red>Upload Attachments(optional)</font>

#### .xml attachment

In [ ]:
xml_attachment = upload_attachment.upload(VATREturnAttachmentURL, "mva-vedlegg.xml", "text/xml" , altinn_token)

#### .pdf attachment

In [ ]:
pdf_attachment = upload_attachment.upload(VATREturnAttachmentURL, "pdf-vedlegg.pdf", "application/pdf", altinn_token)

#### .png attachment

In [ ]:
png_attachment = upload_attachment.upload(VATREturnAttachmentURL, "png-vedlegg.png", "image/png", altinn_token)

## <font color=red>File VAT Return Submission</font>

In [ ]:
FillingProcess = update_process.next_step(instanceUrl, altinn_token)

## <font color=red>Get Instance</font>

In [ ]:
instance = get_instance.get(instanceUrl, altinn_token)